<a href="https://colab.research.google.com/github/GabrielPazz/ImersaoIA-Alura/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn e outras plataformas onde a Alura costuma divulgar seus eventos e cursos.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail no site da Alura para receber informações sobre lançamentos e eventos.
*   **Consultar o site da Alura:** Visite regularmente o site da Alura e procure na seção de cursos e eventos.

Assim que a data da próxima Imersão IA com Google Gemini for definida, a Alura certamente divulgará em seus canais de comunicação.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições para o curso, que é gratuito, podem ser feitas até o dia 11 de maio. A Imersão IA é um curso online com cinco aulas, onde você aprenderá a criar prompts, automatizar tarefas e desenvolver sistemas de agentes autônomos usando o Google Gemini. Ao final do curso, você receberá um certificado Alura + Google.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br, youtube.com



In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [18]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [26]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [27]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes IA
Maravilha! Vamos então criar o post sobre novidades em agentes IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas notícias sobre lançamentos recentes e relevantes relacionados a agentes de IA:
> 
> 
> Com base nas notícias mais recentes, aqui estão alguns lançamentos de agentes de IA que parecem ser relevantes:
> 
> 1.  **IBM watsonx Orchestrate:** A IBM anunciou atualizações para o watsonx Orchestrate, oferecendo ferramentas para integrar, personalizar e implementar agentes de IA, incluindo agentes pré-construídos para RH, vendas e compras.
> 2.  **NVIDIA Modelos Abertos de IA:** A NVIDIA lançou uma família de modelos abertos de IA com capacidade de raciocínio para permitir que desenvolvedores e empresas criem plataformas de agentes de IA. A Deloitte planeja incorporar modelos de raciocínio Llama Nemotron em sua plataforma de IA de agente Zora AI.
> 3.  **Microsoft Copilot Novos Agentes de IA:** A Microsoft anunciou novos agentes de IA para o Microsoft 365 Copilot, chamados Researcher e Analyst, para criar relatórios e analisar dados.
> 4.  **Google Gemini 2.0:** O Google lançou o Gemini 2.0 com capacidades multimodais aprimoradas, incluindo geração nativa de imagens e áudio, abrindo caminho para a "era dos agentes" do Google.
> 5.  **Yellowfin versão 9.15:** A Yellowfin, uma plataforma de business intelligence, apresentou a versão 9.15, que incorpora a funcionalidade de Natural Query Language habilitada por IA, permitindo que os usuários realizem consultas por meio de linguagem natural.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos de agentes de IA que você forneceu, vou criar um plano de conteúdo para cada um, identificar o tema mais relevante e, em seguida, elaborar um plano detalhado para um post sobre esse tema.
> 
> Primeiro, vou usar o Google Search para aprofundar meu conhecimento sobre cada um dos lançamentos:
> 
> 
> Com base nas pesquisas, aqui estão os possíveis temas e planos de conteúdo:
> 
> **1. IBM watsonx Orchestrate:**
> 
> *   **Pontos Relevantes:**
>     *   Atualizações que facilitam a integração, personalização e implementação de agentes de IA.
>     *   Agentes pré-construídos para RH, vendas e compras.
>     *   Introdução do Single Sign-On (SSO) para melhorar a autenticação do usuário durante o uso de aplicativos de terceiros no chat.
>     *   API para a nova experiência de chat baseada em agente.
> *   **Plano de Conteúdo:**
>     *   O que é o IBM watsonx Orchestrate e para que serve?
>     *   Como as atualizações mais recentes beneficiam as empresas?
>     *   Explicação do SSO e seu impacto na segurança e experiência do usuário.
>     *   Exemplos de como os agentes pré-construídos podem ser usados em diferentes departamentos.
>     *   Como construir e implantar agentes usando o Agent Development Kit (ADK).
> 
> **2. NVIDIA Modelos Abertos de IA:**
> 
> *   **Pontos Relevantes:**
>     *   Lançamento de uma família de modelos abertos de IA com capacidade de raciocínio (Llama Nemotron).
>     *   Modelos disponíveis como microsserviços NVIDIA NIM.
>     *   Deloitte incorporando modelos de raciocínio Llama Nemotron em sua plataforma de IA de agente Zora AI.
>     *   Ferramentas e softwares NVIDIA AI Enterprise para agilizar a adoção de raciocínio avançado em sistemas de IA colaborativa.
> *   **Plano de Conteúdo:**
>     *   O que são os modelos abertos de IA da NVIDIA e por que são importantes?
>     *   Como a capacidade de raciocínio desses modelos beneficia os desenvolvedores?
>     *   O que são os microsserviços NVIDIA NIM e como eles otimizam a implantação?
>     *   Exemplos de como empresas como Deloitte e ServiceNow estão usando esses modelos.
>     *   Como usar as ferramentas e softwares NVIDIA AI Enterprise para construir agentes de IA.
> 
> **3. Microsoft Copilot Novos Agentes de IA:**
> 
> *   **Pontos Relevantes:**
>     *   Anúncio de novos agentes de IA para o Microsoft 365 Copilot: Researcher e Analyst.
>     *   Agentes utilizam modelos de IA da OpenAI para análise de dados e pesquisa.
>     *   Researcher: para criar relatórios com informações internas e externas.
>     *   Analyst: para análise de dados com raciocínio em cadeia de pensamentos.
> *   **Plano de Conteúdo:**
>     *   O que são os agentes Researcher e Analyst e como eles se encaixam no Microsoft 365 Copilot?
>     *   Como esses agentes utilizam modelos de IA da OpenAI?
>     *   Exemplos de tarefas que o Researcher e o Analyst podem realizar.
>     *   Benefícios para usuários do Microsoft 365 em termos de produtividade e eficiência.
>     *   Como as organizações podem incorporar o Deep Reasoning em seus agentes.
> 
> **4. Google Gemini 2.0:**
> 
> *   **Pontos Relevantes:**
>     *   Lançamento do Gemini 2.0 com capacidades multimodais aprimoradas (geração nativa de imagens e áudio).
>     *   Modelo projetado para a "era dos agentes" do Google.
>     *   Gemini 2.0 Flash: modelo experimental disponível para desenvolvedores.
>     *   Integração com APIs do Google para tarefas complexas em tempo real.
> *   **Plano de Conteúdo:**
>     *   O que é o Google Gemini 2.0 e como ele se compara às versões anteriores?
>     *   Como as capacidades multimodais aprimoradas abrem caminho para novos agentes de IA?
>     *   Exemplos de aplicações práticas do Gemini 2.0 em diferentes áreas.
>     *   Como desenvolvedores podem acessar e integrar o Gemini 2.0 em seus aplicativos.
>     *   Discussão sobre o impacto esperado do Gemini 2.0 no mercado de IA.
> 
> **5. Yellowfin versão 9.15:**
> 
> *   **Pontos Relevantes:**
>     *   Incorporação da funcionalidade de Natural Query Language (NLQ) habilitada por IA (AI NLQ).
>     *   Usuários podem realizar consultas por meio de linguagem natural.
>     *   AI NLQ simplifica consultas de dados e aumenta a acessibilidade para usuários de negócios.
>     *   Integração perfeita com Yellowfin Dashboards, Presentations e Stories.
> *   **Plano de Conteúdo:**
>     *   O que é o Yellowfin 9.15 e qual o seu principal destaque?
>     *   Como a funcionalidade AI NLQ permite que usuários façam consultas em linguagem natural?
>     *   Benefícios do AI NLQ em termos de acessibilidade, facilidade de uso e geração de insights.
>     *   Como o AI NLQ se integra com outras funcionalidades do Yellowfin?
>     *   Comparativo com outras soluções de NLQ no mercado.
> 
> **Tema Mais Relevante e Plano Detalhado:**
> 
> Com base nas minhas pesquisas, o tema mais relevante para um post seria: **Google Gemini 2.0: A Nova Era dos Agentes de IA**
> 
> *   **Justificativa:** O Google Gemini 2.0 representa um avanço significativo nas capacidades multimodais e no desenvolvimento de agentes de IA. O lançamento gerou bastante expectativa e interesse na comunidade de tecnologia, e um post detalhado sobre suas funcionalidades e aplicações seria muito relevante para o público.
> 
> **Plano Detalhado do Post:**
> 
> 1.  **Título:** Google Gemini 2.0: A Revolução dos Agentes de IA Chegou!
> 2.  **Introdução:**
>     *   Breve introdução sobre a importância dos agentes de IA e sua evolução.
>     *   Anúncio do lançamento do Google Gemini 2.0 e seu impacto no cenário da IA.
>     *   Destaque para o foco do Gemini 2.0 na "era dos agentes".
> 3.  **O que é o Google Gemini 2.0?**
>     *   Explicação detalhada do que é o Gemini 2.0 e suas principais características.
>     *   Comparação com as versões anteriores (Gemini 1.0 e 1.5).
>     *   Destaque para as capacidades multimodais aprimoradas: geração nativa de imagens e áudio.
> 4.  **Funcionalidades e Aplicações:**
>     *   Exemplos de como o Gemini 2.0 pode ser usado em diferentes áreas:
>         *   Criação de conteúdo multimodal (texto, imagem, áudio).
>         *   Automação de tarefas complexas.
>         *   Assistência virtual avançada.
>         *   Desenvolvimento de jogos.
>     *   Integração com APIs do Google (Search, Lens, Maps) para tarefas em tempo real.
> 5.  **Gemini 2.0 Flash:**
>     *   Apresentação do Gemini 2.0 Flash como um modelo experimental para desenvolvedores.
>     *   Destaque para a velocidade e eficiência do Gemini 2.0 Flash.
>     *   Como desenvolvedores podem acessar e integrar o Gemini 2.0 Flash em seus aplicativos.
> 6.  **O Futuro dos Agentes de IA com Gemini 2.0:**
>     *   Discussão sobre o impacto esperado do Gemini 2.0 no mercado de IA.
>     *   Como o Gemini 2.0 pode influenciar o desenvolvimento de assistentes virtuais mais inteligentes e autônomos.
>     *   Projetos futuros do Google na área de agentes de IA (Astra e Mariner).
> 7.  **Conclusão:**
>     *   Reafirmação da importância do Google Gemini 2.0 como um marco na era dos agentes de IA.
>     *   Incentivo para que os leitores explorem e experimentem as funcionalidades do Gemini 2.0.
> 8.  **Chamada para Ação:**
>     *   Convite para que os leitores deixem seus comentários e compartilhem suas opiniões sobre o Gemini 2.0.
>     *   Links para recursos adicionais e documentação do Google Gemini 2.0.
> 
> Este plano detalhado fornece uma estrutura abrangente para um post informativo e envolvente sobre o Google Gemini 2.0 e seu papel na evolução dos agentes de IA.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Okay, Agente Redator! Com o plano detalhado do Google Gemini 2.0 em mãos, vamos criar um rascunho de post para o Instagram da Alura, focado em engajamento e informação.
> 
> ---
> 
> **Rascunho de Post para Instagram:**
> 
> Prepare-se! 🤖🚀 A era dos Agentes de IA acaba de dar um passo GIGANTE com o lançamento do **Google Gemini 2.0**!
> 
> Esqueça a IA que só entende texto. O Gemini 2.0 chegou turbinado, feito sob medida para essa nova fase da inteligência artificial, onde os "agentes" são protagonistas e a interação é muito mais rica.
> 
> 👉 **Principais Novidades que você PRECISA saber:**
> 
> *   **Multimodal de Verdade:** Ele agora *gera* imagens e áudio nativamente, abrindo um leque ENORME de possibilidades para criação de conteúdo e experiências mais imersivas! 🤯
> *   **Foco nos Agentes:** Projetado pensando em como a IA pode atuar de forma mais autônoma e integrada em tarefas complexas, como um verdadeiro "agente" digital.
> *   **Gemini 2.0 Flash:** Tem uma versão experimental focada em velocidade e eficiência, perfeita para desenvolvedores que querem inovar e colocar a mão na massa AGORA!
> 
> Imagina assistentes virtuais que criam o resumo da sua reunião em áudio, ou IAs que geram assets visuais para seu projeto enquanto você conversa com elas? É pra lá que estamos indo!
> 
> O Google Gemini 2.0 não é só uma atualização, é um vislumbre do futuro dos agentes de IA e do que podemos construir com essa tecnologia.
> 
> O que você achou dessa novidade? Como imagina os agentes de IA transformando o nosso dia a dia? Deixa seu comentário! 👇
> 
> #InteligenciaArtificial #GoogleGemini #AgentesIA #Inovacao


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
